In [ ]:
!pip install -q -U google-genai

In [ ]:
import pandas as pd

contexts_df = pd.read_csv("gic_data_contexts_final_with_meta.csv")
print(len(contexts_df))
contexts_df.head(1)

## Stratified Random Sampling for GIC RAG Dataset Creation

To create a representative subset for our RAG experiments, we performed proportional stratified sampling based on the main service categories.  
This ensures that the 500 sampled contexts preserve the distribution of main categories from the original dataset of 1,699 contexts.  

A fixed random seed (`random_state=42`) was used to guarantee reproducibility.  
Rounding adjustments were applied to ensure the total number of sampled contexts equals exactly 500 while maintaining category proportions.

In [ ]:
import numpy as np

MAX_SIZE = 500

# Calculate target sample sizes per category
group_sizes = contexts_df.groupby("main_category_name").size()
proportions = group_sizes / group_sizes.sum()

target_sizes = (proportions * MAX_SIZE).round().astype(int)

# Fix rounding drift
difference = MAX_SIZE - target_sizes.sum()

if difference != 0:
    adjustment = target_sizes.sort_values(ascending=False).index[:abs(difference)]
    target_sizes.loc[adjustment] += np.sign(difference)

# Sample exactly
sampled_df = (
    contexts_df
    .groupby("main_category_name", group_keys=False)
    .apply(lambda x: x.sample(
        n=target_sizes.loc[x.name],
        random_state=42
    ))
)

len(sampled_df)


/tmp/ipython-input-1461358329.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


500

In [ ]:
sampled_df.head()

,context_id,context_title,context_content,main_category_id,main_category_name,sub_category_name,sub_category_id,service_name,service_id,service_link,context_with_metadata
965,8-47-1776-2,Contact Information and Additional Notes,Staff Officers in charge of the services:\n\nD...,8,"Agriculture, Livestock & Fisheries","Agriculture, Livestock & Fisheries Development",47,09.Approval for importation of Poultry meat an...,1776,https://gic.gov.lk/gic/index.php/en/component/...,Service: 09.Approval for importation of Poultr...
938,8-47-817-2,Contact Information and Additional Information,Position | Name | Division | Tel. No. | Fax | ...,8,"Agriculture, Livestock & Fisheries","Agriculture, Livestock & Fisheries Development",47,Obtaining of compost and agricultural plants,817,https://gic.gov.lk/gic/index.php/en/component/...,Service: Obtaining of compost and agricultural...
841,8-44-927-3,Additional Notes and Timeline,Note 1: If the inspector does not indicate app...,8,"Agriculture, Livestock & Fisheries",Approval & Registration,44,"Registration of Manufacturers of weights, Meas...",927,https://gic.gov.lk/gic/index.php/en/component/...,Service: Registration of Manufacturers of weig...
900,8-47-1320-2,Eligibility and Registration Procedure (Part 2),"After harvesting, a sample will be drawn for q...",8,"Agriculture, Livestock & Fisheries","Agriculture, Livestock & Fisheries Development",47,SCPPC-Seed Certification: Registration of seed...,1320,https://gic.gov.lk/gic/index.php/en/component/...,Service: SCPPC-Seed Certification: Registratio...
980,8-48-1350-2,Contact Information for Inter-Provincial Agric...,Staff officers in charge of the service:\n\nIn...,8,"Agriculture, Livestock & Fisheries",Training & Extension Services,48,Extension &Training: Agriculture extension ser...,1350,https://gic.gov.lk/gic/index.php/en/component/...,Service: Extension &Training: Agriculture exte...


In [ ]:
len(sampled_df)

500

In [ ]:
orig_dist = contexts_df["main_category_name"].value_counts()
sample_dist = sampled_df["main_category_name"].value_counts()

dist_df = (
    orig_dist
    .rename("original_count")
    .to_frame()
    .join(sample_dist.rename("sampled_count"))
    .fillna(0)
)

dist_df

,original_count,sampled_count
main_category_name,,
"Trade, Business & Industry",419,124
"Agriculture, Livestock & Fisheries",197,59
"Health, Well Being & Social Service",168,49
"Justice, Law & Rights",147,43
"Banking, Tax & Insurance",113,33
Education & Training,110,32
Employment Information,107,31
"Housing, Property & Utilities",92,27
Citizen's Registrations,89,26


In [ ]:
dist_df["original_pct"] = dist_df["original_count"] / len(contexts_df) * 100
dist_df["sampled_pct"] = dist_df["sampled_count"] / len(sampled_df) * 100

dist_df[["original_pct", "sampled_pct"]]

,original_pct,sampled_pct
main_category_name,,
"Trade, Business & Industry",24.661566,24.8
"Agriculture, Livestock & Fisheries",11.595056,11.8
"Health, Well Being & Social Service",9.888170,9.8
"Justice, Law & Rights",8.652148,8.6
"Banking, Tax & Insurance",6.650971,6.6
Education & Training,6.474397,6.4
Employment Information,6.297822,6.2
"Housing, Property & Utilities",5.414950,5.4
Citizen's Registrations,5.238376,5.2


In [ ]:
dist_df[["original_pct", "sampled_pct"]].plot(
    kind="bar",
    figsize=(12, 5),
    title="Main Category Distribution: Original vs Sampled"
)

In [ ]:
from scipy.stats import chisquare

chisquare(
    f_obs=dist_df["sampled_count"],
    f_exp=dist_df["original_count"] * (len(sampled_df) / len(contexts_df))
)

In [ ]:
sampled_df.to_csv("gic_data_contexts_final_with_meta_stratified_sample.csv", index=False)

In [ ]:
len(sampled_df)

500

## Generate questions

In [ ]:
from google import genai
import json
import time
import pandas as pd

client = genai.Client(api_key="")

In [ ]:
contexts = pd.read_csv("gic_data_contexts_final_with_meta_stratified_sample.csv")
contexts.head()

,context_id,context_title,context_content,main_category_id,main_category_name,sub_category_name,sub_category_id,service_name,service_id,service_link,context_with_metadata
0,8-47-1776-2,Contact Information and Additional Notes,Staff Officers in charge of the services:\n\nD...,8,"Agriculture, Livestock & Fisheries","Agriculture, Livestock & Fisheries Development",47,09.Approval for importation of Poultry meat an...,1776,https://gic.gov.lk/gic/index.php/en/component/...,Service: 09.Approval for importation of Poultr...
1,8-47-817-2,Contact Information and Additional Information,Position | Name | Division | Tel. No. | Fax | ...,8,"Agriculture, Livestock & Fisheries","Agriculture, Livestock & Fisheries Development",47,Obtaining of compost and agricultural plants,817,https://gic.gov.lk/gic/index.php/en/component/...,Service: Obtaining of compost and agricultural...
2,8-44-927-3,Additional Notes and Timeline,Note 1: If the inspector does not indicate app...,8,"Agriculture, Livestock & Fisheries",Approval & Registration,44,"Registration of Manufacturers of weights, Meas...",927,https://gic.gov.lk/gic/index.php/en/component/...,Service: Registration of Manufacturers of weig...
3,8-47-1320-2,Eligibility and Registration Procedure (Part 2),"After harvesting, a sample will be drawn for q...",8,"Agriculture, Livestock & Fisheries","Agriculture, Livestock & Fisheries Development",47,SCPPC-Seed Certification: Registration of seed...,1320,https://gic.gov.lk/gic/index.php/en/component/...,Service: SCPPC-Seed Certification: Registratio...
4,8-48-1350-2,Contact Information for Inter-Provincial Agric...,Staff officers in charge of the service:\n\nIn...,8,"Agriculture, Livestock & Fisheries",Training & Extension Services,48,Extension &Training: Agriculture extension ser...,1350,https://gic.gov.lk/gic/index.php/en/component/...,Service: Extension &Training: Agriculture exte...


In [ ]:
contexts.iloc[76]

,76
context_id,3-24-803-2
context_title,Required Documents for Savings Certificates an...
context_content,Savings Certificates Required Documents\n\n- [...
main_category_id,3
main_category_name,"Banking, Tax & Insurance"
sub_category_name,Bank Accounts
sub_category_id,24
service_name,Term Deposits (Fixed Deposit)
service_id,803
service_link,https://gic.gov.lk/gic/index.php/en/component/...


In [ ]:
data = []
start_index = 76
i = start_index
for index, context in contexts.iterrows():
  if index < start_index:
    continue
  if index != 0 and index % 10 == 0:
    time.sleep(1.5)
  print("processing: ", index)
  prompt = """
  You are given a context in english.
  Your task is to generate question and answer pairs from the given context in language english, tamil and sinhala.
  Your question should be meaningfull and reflect the meta data of the context.

  When generating a question in particluar language, the question should be in that language only, Dont include words from other languages or unicode character.

  You have return the result as below json object.
  {
    "question_en": "question in english",
    "answer_en": "answer in english",
    "question_ta": "question in tamil",
    "answer_ta": "answer in tamil",
    "question_si": "question in sinhala",
    "answer_si": "answer in sinhala"
  }

  context:

  """ + context['context_with_metadata']

  isResponseGenearted = False
  MAX_RETRIES = 5
  for attempt in range(5):
        try:
            response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=prompt,
            )
            isResponseGenearted = True
            print("QA generation successful")
            break
        except ServerError as e:
            wait = 3 * (attempt + 1)
            print(f"Server error: [{type(e).__name__}]: {e} \nRetry {attempt + 1}/{5} in {wait}s")
            time.sleep(wait)
        except Exception as e:
            print(f"Unknown error occurred [{type(e).__name__}]: {e}")
            break

  if isResponseGenearted:
    try:
      ans = response.candidates[0].content.parts[0].text.split("```")[1].split("json")[1]
      jans = json.loads(ans)
      data.append(
          {
            "context_id": context['context_id'],
            "context": context['context_with_metadata'],
            "question_en": jans["question_en"],
            "answer_en": jans["answer_en"],
            "question_ta": jans["question_ta"],
            "answer_ta": jans["answer_ta"],
            "question_si": jans["question_si"],
            "answer_si": jans["answer_si"]
          }
      )
    except Exception as e:
      print("JSON parse error: ", response)
      data.append(
        {
          "context_id": context['context_id'],
          "context": context['context_with_metadata'],
          "question_en": "",
          "answer_en": "",
          "question_ta": "",
          "answer_ta": "",
          "question_si": "",
          "answer_si": ""
        }
      )
      break
  else:
    data.append(
        {
          "context_id": context['context_id'],
          "context": context['context_with_metadata'],
          "question_en": "",
          "answer_en": "",
          "question_ta": "",
          "answer_ta": "",
          "question_si": "",
          "answer_si": ""
        }
    )
  print("data length: ", len(data))
  i+=1

qa_data = pd.DataFrame(data)
qa_data.head()

processing:  76
QA generation successful
data length:  1
processing:  77
QA generation successful
data length:  2
processing:  78
QA generation successful
data length:  3
processing:  79
QA generation successful
data length:  4
processing:  80
QA generation successful
data length:  5
processing:  81
QA generation successful
data length:  6
processing:  82
QA generation successful
data length:  7
processing:  83
QA generation successful
data length:  8
processing:  84
QA generation successful
data length:  9
processing:  85
QA generation successful
data length:  10
processing:  86
QA generation successful
data length:  11
processing:  87
QA generation successful
data length:  12
processing:  88
QA generation successful
data length:  13
processing:  89
QA generation successful
data length:  14
processing:  90
QA generation successful
data length:  15
processing:  91
QA generation successful
data length:  16
processing:  92
QA generation successful
data length:  17
processing:  93
QA gene

,context_id,context,question_en,answer_en,question_ta,answer_ta,question_si,answer_si
0,3-24-803-2,Service: Term Deposits (Fixed Deposit)\nMain C...,What is the approximate time duration for open...,The approximate time duration for opening an a...,சேமிப்பு சான்றிதழ்கள் அல்லது திறைசேரி பில்களுக...,சேமிப்பு சான்றிதழ்கள் அல்லது திறைசேரி பில்களுக...,ඉතිරිකිරීමේ සහතික හෝ භාණ්ඩාගාර බිල්පත් සඳහා ගි...,ඉතිරිකිරීමේ සහතික හෝ භාණ්ඩාගාර බිල්පත් සඳහා ගි...
1,3-25-811-1,Service: Obtain Small & Medium Loans\nMain Cat...,What are the eligibility criteria for Gamata N...,Gamata Naya Enterprises operating outside Colo...,"சிறிய மற்றும் நடுத்தர கடன்களைப் பெறுவதற்கு ""கம...",கொழும்பு மற்றும் கம்பஹா மாவட்டங்களுக்கு வெளியே...,කුඩා හා මධ්‍ය පරිමාණ ණය ලබා ගැනීම සඳහා ගමට ණය ...,කොළඹ සහ ගම්පහ දිස්ත්‍රික්කවලින් පිටත ක්‍රියාත්...
2,3-24-2367-1,"Service: RDB Accounts\nMain Category: Banking,...",What is the specific topic related to RDB Acco...,The topic is 'Related Services and Buying Publ...,வங்கிக் கணக்குகள் துணைப்பிரிவின் கீழ் RDB கணக்...,தலைப்பு 'தொடர்புடைய சேவைகள் மற்றும் வெளியீடுகள...,බැංකු ගිණුම් උපප්‍රවර්ගය යටතේ RDB ගිණුම් සම්බන...,මාතෘකාව 'අදාළ සේවා සහ ප්‍රකාශන මිලදී ගැනීම' වේ.
3,3-26-1156-7,Service: GUIDELINES AND CONDITIONS APPLICABLE ...,What are the additional requirements for obtai...,"For Club Licences, applicants must submit a Ce...",கிளப் உரிமங்களைப் பெறுவதற்கான மேலதிகத் தேவைகள்...,"கிளப் உரிமங்களுக்கு, விண்ணப்பதாரர்கள் 1975 இன்...",ක්ලබ් බලපත්‍ර ලබා ගැනීම සඳහා අමතර අවශ්‍යතා මොන...,"ක්ලබ් බලපත්‍ර සඳහා, අයදුම්කරුවන් 1975 අංක 17 ද..."
4,3-25-1860-3,Service: Small & Medium Scale Loans\nMain Cate...,What are the guarantor requirements for small ...,For small and medium-scale loans below Rs. 100...,"ரூ. 100,000/- க்கும் குறைவான சிறிய மற்றும் நடு...","ரூ. 100,000/- க்கும் குறைவான சிறிய மற்றும் நடு...","රු. 100,000/- ට අඩු සුළු හා මධ්‍ය පරිමාණ ණය සඳ...","රු. 100,000/- ට අඩු සුළු හා මධ්‍ය පරිමාණ ණය සඳ..."


In [ ]:
qa_data = pd.DataFrame(data)
qa_data.head()

,context_id,context,question_en,answer_en,question_ta,answer_ta,question_si,answer_si
0,3-24-803-2,Service: Term Deposits (Fixed Deposit)\nMain C...,What is the approximate time duration for open...,The approximate time duration for opening an a...,சேமிப்பு சான்றிதழ்கள் அல்லது திறைசேரி பில்களுக...,சேமிப்பு சான்றிதழ்கள் அல்லது திறைசேரி பில்களுக...,ඉතිරිකිරීමේ සහතික හෝ භාණ්ඩාගාර බිල්පත් සඳහා ගි...,ඉතිරිකිරීමේ සහතික හෝ භාණ්ඩාගාර බිල්පත් සඳහා ගි...
1,3-25-811-1,Service: Obtain Small & Medium Loans\nMain Cat...,What are the eligibility criteria for Gamata N...,Gamata Naya Enterprises operating outside Colo...,"சிறிய மற்றும் நடுத்தர கடன்களைப் பெறுவதற்கு ""கம...",கொழும்பு மற்றும் கம்பஹா மாவட்டங்களுக்கு வெளியே...,කුඩා හා මධ්‍ය පරිමාණ ණය ලබා ගැනීම සඳහා ගමට ණය ...,කොළඹ සහ ගම්පහ දිස්ත්‍රික්කවලින් පිටත ක්‍රියාත්...
2,3-24-2367-1,"Service: RDB Accounts\nMain Category: Banking,...",What is the specific topic related to RDB Acco...,The topic is 'Related Services and Buying Publ...,வங்கிக் கணக்குகள் துணைப்பிரிவின் கீழ் RDB கணக்...,தலைப்பு 'தொடர்புடைய சேவைகள் மற்றும் வெளியீடுகள...,බැංකු ගිණුම් උපප්‍රවර්ගය යටතේ RDB ගිණුම් සම්බන...,මාතෘකාව 'අදාළ සේවා සහ ප්‍රකාශන මිලදී ගැනීම' වේ.
3,3-26-1156-7,Service: GUIDELINES AND CONDITIONS APPLICABLE ...,What are the additional requirements for obtai...,"For Club Licences, applicants must submit a Ce...",கிளப் உரிமங்களைப் பெறுவதற்கான மேலதிகத் தேவைகள்...,"கிளப் உரிமங்களுக்கு, விண்ணப்பதாரர்கள் 1975 இன்...",ක්ලබ් බලපත්‍ර ලබා ගැනීම සඳහා අමතර අවශ්‍යතා මොන...,"ක්ලබ් බලපත්‍ර සඳහා, අයදුම්කරුවන් 1975 අංක 17 ද..."
4,3-25-1860-3,Service: Small & Medium Scale Loans\nMain Cate...,What are the guarantor requirements for small ...,For small and medium-scale loans below Rs. 100...,"ரூ. 100,000/- க்கும் குறைவான சிறிய மற்றும் நடு...","ரூ. 100,000/- க்கும் குறைவான சிறிய மற்றும் நடு...","රු. 100,000/- ට අඩු සුළු හා මධ්‍ය පරිමාණ ණය සඳ...","රු. 100,000/- ට අඩු සුළු හා මධ්‍ය පරිමාණ ණය සඳ..."


In [ ]:
len(qa_data)

424

In [ ]:
qa_data.tail()

,context_id,context,question_en,answer_en,question_ta,answer_ta,question_si,answer_si
419,4-91-643-1,Service: State Music Ensemble\nMain Category: ...,Who provides the services of the State Music E...,The Department of Cultural provides the servic...,அரசு இசைக்குழு சேவைகளை யார் வழங்குகிறார்கள் மற...,கலாச்சார திணைக்களம் அரசு இசைக்குழு சேவைகளை வழங...,රාජ්‍ය සංගීත කණ්ඩායම් සේවා සපයන්නේ කවුද සහ රාජ...,සංස්කෘතික දෙපාර්තමේන්තුව රාජ්‍ය සංගීත කණ්ඩායම්...
420,4-97-1481-2,Service: Museum Publications\nMain Category: T...,What is the specific topic addressed by the pu...,The specific topic addressed by the publicatio...,தேசிய அருங்காட்சியகங்களின் வெளியீடுகளின் பட்டி...,வெளியீடுகளின் பட்டியலில் குறிப்பிடப்பட்டுள்ள க...,ජාතික කෞතුකාගාර ප්‍රකාශන ලැයිස්තුවෙන් ආමන්ත්‍ර...,ප්‍රකාශන ලැයිස්තුවෙන් ආමන්ත්‍රණය කරන නිශ්චිත ම...
421,4-91-638-1,Service: State Arts and Sculpture Contest & F...,Who is eligible to submit creations for the St...,Any person or artiste over 18 years who are en...,அரச கலை மற்றும் சிற்பப் போட்டி மற்றும் விழாவிற...,இலங்கையில் படைப்புப் பணிகளில் ஈடுபட்டுள்ள 18 வ...,රාජ්‍ය කලා හා මූර්ති තරගාවලිය සහ උත්සවය සඳහා න...,ශ්‍රී ලංකාවේ නිර්මාණාත්මක කටයුතුවල නිරත වන වයස...
422,4-91-955-1,Service: Visit to Haldummulla Herbal Garden\nM...,Who is allowed to visit the Haldummulla Herbal...,Any person is allowed to visit the Haldummulla...,ஹல்தும்முல்ல மூலிகை பூங்காவிற்கு யார் பார்வையி...,ஹல்தும்முல்ல மூலிகை பூங்காவிற்கு யார் வேண்டுமா...,"හල්දුම්මුල්ල ඖෂධ උයන නැරඹීමට අවසර ඇත්තේ කාටද, ...",ඕනෑම අයෙකුට හල්දුම්මුල්ල ඖෂධ උයන නැරඹීමට අවසර ...
423,4-27-653-2,Service: National Council for Road Safety\nMai...,What are the specific conditions under which c...,Compensation is paid only in instances where a...,அடையாளம் தெரியாத வாகனங்களால் ஏற்படும் விபத்துக...,அடையாளம் தெரியாத வாகனம் சம்பந்தப்பட்ட விபத்தில...,හඳුනා නොගත් වාහන නිසා සිදුවන අනතුරුවලට ලක්වන ප...,හඳුනා නොගත් වාහනයක් සම්බන්ධ රිය අනතුරකින් පුද්...


In [ ]:
qa_data.to_csv(f"gic_qa_76_499.csv", index=False)


In [ ]:
df1 = pd.read_csv("gic_qa_0_1.csv")
df2 = pd.read_csv("gic_qa_1_75.csv")
df3 = pd.read_csv("gic_qa_76_499.csv")

qa_data = pd.concat([df1, df2, df3])
len(qa_data)

500

In [ ]:
qa_data.to_csv("gic_qa_0_499.csv", index=False)

In [ ]:
import pandas as pd

cols = [
    "context_id",
    "context",
    "question_en", "answer_en",
    "question_ta", "answer_ta",
    "question_si", "answer_si"
]

empty_counts = {}

for col in cols:
    empty_counts[col] = (
        qa_data[col].isna() |
        (qa_data[col].astype(str).str.strip() == "")
    ).sum()

pd.DataFrame.from_dict(
    empty_counts, orient="index", columns=["empty_count"]
)


,empty_count
context_id,0
context,0
question_en,0
answer_en,0
question_ta,0
answer_ta,0
question_si,0
answer_si,0


In [ ]:
qa_data = qa_data.reset_index(drop=True)
qa_data["question_id"] = range(1, len(qa_data) + 1)
print(len(qa_data))

In [ ]:
qa_data.head()

In [ ]:
qa_data.tail()

In [ ]:
qa_data.to_csv("gic_qa_with_ids.csv", index=False)